In [1]:
#2.	Find the PriceGain, PriceGainP, VolumeGain, VolumeGainP of each stock for 10 days [Refer Point 1] from historical data, write the results to JSON, Parquet, ORC, 

import findspark
findspark.init()

In [2]:
from pyspark.conf import SparkConf
config = SparkConf()
config.setMaster("local").setAppName("DataFrameAsg02")

from pyspark.sql import SparkSession
spark = SparkSession.builder.config(conf=config).getOrCreate()

sc= spark.sparkContext

22/03/12 05:23:37 WARN Utils: Your hostname, ubuntu-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.80.128 instead (on interface ens33)
22/03/12 05:23:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/03/12 05:23:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
#SYMBOL,SERIES,OPEN,HIGH,LOW,CLOSE,LAST,PREVCLOSE,TOTTRDQTY,TOTTRDVAL,TIMESTAMP,TOTALTRADES,ISIN,
from pyspark.sql.types import StructType, LongType, StringType, IntegerType, DoubleType, DateType, TimestampType

stockSchema = StructType()\
                    .add("SYMBOL", StringType(),True)\
                    .add("SERIES", StringType(),True)\
                    .add("OPEN", DoubleType(),True)\
                    .add("HIGH", DoubleType(),True)\
                    .add("LOW", DoubleType(),True)\
                    .add("CLOSE", DoubleType(),True)\
                    .add("LAST", DoubleType(),True)\
                    .add("PREVCLOSE", DoubleType(),True)\
                    .add("TOTTRDQTY", IntegerType(),True)\
                    .add("TOTTRDVAL", DoubleType(),True)\
                    .add("TIMESTAMP", StringType(),True)\
                    .add("TOTALTRADES", IntegerType(),True)\
                    .add("ISIN", StringType(),True)                   

In [4]:
stockDf= spark.read.format("csv").option("header", True).schema(stockSchema).load("hdfs://localhost:9000/asgStocks")
stockDf.printSchema()

stockDf.show(3)

root
 |-- SYMBOL: string (nullable = true)
 |-- SERIES: string (nullable = true)
 |-- OPEN: double (nullable = true)
 |-- HIGH: double (nullable = true)
 |-- LOW: double (nullable = true)
 |-- CLOSE: double (nullable = true)
 |-- LAST: double (nullable = true)
 |-- PREVCLOSE: double (nullable = true)
 |-- TOTTRDQTY: integer (nullable = true)
 |-- TOTTRDVAL: double (nullable = true)
 |-- TIMESTAMP: string (nullable = true)
 |-- TOTALTRADES: integer (nullable = true)
 |-- ISIN: string (nullable = true)



22/03/12 05:23:53 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 14, schema size: 13
CSV file: hdfs://localhost:9000/asgStocks/cm21FEB2022bhav.csv


+----------+------+----+-----+-----+-----+-----+---------+---------+-------------+-----------+-----------+------------+
|    SYMBOL|SERIES|OPEN| HIGH|  LOW|CLOSE| LAST|PREVCLOSE|TOTTRDQTY|    TOTTRDVAL|  TIMESTAMP|TOTALTRADES|        ISIN|
+----------+------+----+-----+-----+-----+-----+---------+---------+-------------+-----------+-----------+------------+
| 20MICRONS|    EQ|76.0|77.55|73.25| 74.1| 73.6|     77.2|   141872|1.061998695E7|21-FEB-2022|       2443|INE144J01027|
|21STCENMGM|    EQ|33.3| 33.3| 33.3| 33.3| 33.3|    33.95|      415|      13819.5|21-FEB-2022|         24|INE253B01015|
| 3IINFOLTD|    EQ|62.0| 63.4|59.05|59.05|59.05|    62.15|   753815| 4.47928636E7|21-FEB-2022|       4999|INE748C01038|
+----------+------+----+-----+-----+-----+-----+---------+---------+-------------+-----------+-----------+------------+
only showing top 3 rows



In [5]:
from pyspark.sql.functions import col, to_timestamp
#price Gain
priceGainDf = stockDf.withColumn("priceGain", col("CLOSE")-col("OPEN"))\
                     .withColumn("PriceGainP", (col("priceGain") / col("OPEN")) * 100)\
                     .withColumn("Date",to_timestamp(("TIMESTAMP"),"dd-MMM-yyyy"))
                  
priceGainDf.printSchema()
priceGainDf.show(5)

root
 |-- SYMBOL: string (nullable = true)
 |-- SERIES: string (nullable = true)
 |-- OPEN: double (nullable = true)
 |-- HIGH: double (nullable = true)
 |-- LOW: double (nullable = true)
 |-- CLOSE: double (nullable = true)
 |-- LAST: double (nullable = true)
 |-- PREVCLOSE: double (nullable = true)
 |-- TOTTRDQTY: integer (nullable = true)
 |-- TOTTRDVAL: double (nullable = true)
 |-- TIMESTAMP: string (nullable = true)
 |-- TOTALTRADES: integer (nullable = true)
 |-- ISIN: string (nullable = true)
 |-- priceGain: double (nullable = true)
 |-- PriceGainP: double (nullable = true)
 |-- Date: timestamp (nullable = true)

+----------+------+-------+--------+--------+-------+-------+---------+---------+-------------+-----------+-----------+------------+-------------------+-------------------+-------------------+
|    SYMBOL|SERIES|   OPEN|    HIGH|     LOW|  CLOSE|   LAST|PREVCLOSE|TOTTRDQTY|    TOTTRDVAL|  TIMESTAMP|TOTALTRADES|        ISIN|          priceGain|         PriceGainP|      

22/03/12 05:24:17 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 14, schema size: 13
CSV file: hdfs://localhost:9000/asgStocks/cm21FEB2022bhav.csv


In [6]:
from pyspark.sql.window import Window
from pyspark.sql.functions import lag, desc

windowspec = Window.partitionBy("SYMBOL").orderBy("DATE")

priceGainDf1 = priceGainDf.withColumn("lag", lag("TOTTRDQTY",1).over(windowspec))\
                          .na.fill(value=0,subset=["lag"])\
                          .withColumn("VolumeGain",col("TOTTRDQTY") - col("lag") )\
                          .withColumn("VolumeGainP",col("VolumeGain") / col("lag") * 100)\
                          .drop("ISIN", "OPEN", "SERIES", "HIGH", "LOW", "CLOSE", "LAST", "PREVCLOSE", "TOTTRDQTY", "TOTTRDVAL", "TIMESTAMP","TOTALTRADES")\
                          .sort(desc("PriceGainP"))

priceGainDf1.show(10)

+----------+-------------------+------------------+-------------------+------+----------+------------------+
|    SYMBOL|          priceGain|        PriceGainP|               Date|   lag|VolumeGain|       VolumeGainP|
+----------+-------------------+------------------+-------------------+------+----------+------------------+
|SREIBNPNCD|               40.0|              32.0|2022-02-25 00:00:00|    74|        91|122.97297297297298|
|      RHFL|  67.69999999999999|28.771780705482357|2022-02-25 00:00:00|683740|   -683196| -99.9204375932372|
|    FELDVR| 2.3499999999999996| 22.59615384615384|2022-02-28 00:00:00|202636|      8218| 4.055547878955369|
|  MODISNME| 13.399999999999999|22.558922558922557|2022-02-28 00:00:00| 46827|     15539| 33.18384692591881|
|  STEELCAS|  53.30000000000001| 22.39495798319328|2022-02-22 00:00:00|  1941|      1792|  92.3235445646574|
|  FILDF2GP|0.21999999999999997| 22.22222222222222|2022-02-25 00:00:00|  1042|      1835| 176.1036468330134|
|KOTHARIPRO|  19.35

In [8]:
priceGainDf1.write.mode("overwrite")\
                    .json("hdfs://localhost:9000/Asg2VolumeGain-json")

priceGainDf1.write.mode("overwrite")\
                    .parquet("hdfs://localhost:9000/Asg2VolumeGain-parquet")

priceGainDf1.write.mode("overwrite")\
                    .orc("hdfs://localhost:9000/Asg2VolumeGain-orc")

priceGainDf1.write.mode("overwrite")\
                    .csv("hdfs://localhost:9000/Asg2VolumeGain-csv")